# Inverted Pendulum
This code is based on [the inverted pendulum example](https://github.com/stack-of-tasks/pinocchio/blob/master/examples/simulation-inverted-pendulum.py) provided by the [Pinocchio](https://stack-of-tasks.github.io/pinocchio/) library. The goal is to balance a pendulum in the upright position by applying a torque to the joint.

The state of the system $x=[\theta\:\:\:\dot{\theta}]$ is described by the angle of the pendulum $\theta$ and its angular velocity $\dot{\theta}$. The control input $u$ is the torque applied to the joint.

In [9]:
import math
import sys
import time

import hppfcl as fcl
import numpy as np
import pinocchio as pin
from pinocchio.visualize import MeshcatVisualizer
import meshcat

import ilqr

In [10]:
N = 1  # number of pendulums
model = pin.Model()
geom_model = pin.GeometryModel()

parent_id = 0
joint_placement = pin.SE3.Identity()
body_mass = 1.0
body_radius = 0.1

shape0 = fcl.Sphere(body_radius)
geom0_obj = pin.GeometryObject("base", 0, shape0, pin.SE3.Identity())
geom0_obj.meshColor = np.array([1.0, 0.1, 0.1, 1.0])
geom_model.addGeometryObject(geom0_obj)

/var/folders/pr/wxvx7wlx1r5fczqp1p8qmmr80000gp/T/ipykernel_4792/2405696514.py:11: UserWarning: This function has been marked as deprecated and will be removed in a future release.
  geom0_obj = pin.GeometryObject("base", 0, shape0, pin.SE3.Identity())


0

In [11]:
for k in range(N):
    joint_name = "joint_" + str(k + 1)
    joint_id = model.addJoint(
        parent_id, pin.JointModelRY(), joint_placement, joint_name
    )

    body_inertia = pin.Inertia.FromSphere(body_mass, body_radius)
    body_placement = joint_placement.copy()
    body_placement.translation[2] = 1.0
    model.appendBodyToJoint(joint_id, body_inertia, body_placement)

    geom1_name = "ball_" + str(k + 1)
    shape1 = fcl.Sphere(body_radius)
    geom1_obj = pin.GeometryObject(geom1_name, joint_id, shape1, body_placement)
    geom1_obj.meshColor = np.ones(4)
    geom_model.addGeometryObject(geom1_obj)

    geom2_name = "bar_" + str(k + 1)
    shape2 = fcl.Cylinder(body_radius / 4.0, body_placement.translation[2])
    shape2_placement = body_placement.copy()
    shape2_placement.translation[2] /= 2.0

    geom2_obj = pin.GeometryObject(geom2_name, joint_id, shape2, shape2_placement)
    geom2_obj.meshColor = np.array([0.0, 0.0, 0.0, 1.0])
    geom_model.addGeometryObject(geom2_obj)

    parent_id = joint_id
    joint_placement = body_placement.copy()


visual_model = geom_model
viz = MeshcatVisualizer(model, geom_model, visual_model)

/var/folders/pr/wxvx7wlx1r5fczqp1p8qmmr80000gp/T/ipykernel_4792/623485344.py:14: UserWarning: This function has been marked as deprecated and will be removed in a future release.
  geom1_obj = pin.GeometryObject(geom1_name, joint_id, shape1, body_placement)
/var/folders/pr/wxvx7wlx1r5fczqp1p8qmmr80000gp/T/ipykernel_4792/623485344.py:23: UserWarning: This function has been marked as deprecated and will be removed in a future release.
  geom2_obj = pin.GeometryObject(geom2_name, joint_id, shape2, shape2_placement)


In [12]:
visual_model = geom_model
viz = MeshcatVisualizer(model, geom_model, visual_model)

# Initialize the viewer.
try:
    viz.initViewer(loadModel=True, viewer=meshcat.Visualizer())
except ImportError as err:
    print(
        "Error while initializing the viewer. "
        "It seems you should install gepetto-viewer"
    )
    print(err)
    sys.exit(0)

try:
    viz.loadViewerModel("pinocchio")
except AttributeError as err:
    print(
        "Error while loading the viewer model. "
        "It seems you should start gepetto-viewer"
    )
    print(err)
    sys.exit(0)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7009/static/


In [ ]:
dt = 0.01
T = 5
N = math.floor(T / dt)

model.lowerPositionLimit.fill(-math.pi)
model.upperPositionLimit.fill(+math.pi)

# initial state
q = pin.randomConfiguration(model)
v = np.zeros(model.nv)
viz.display(q)
q, v = q.item(), v.item()

data_sim = model.createData()

# we define a dynamics function to pass the solver
def dynamics(x, u):
    q, v = x

    # due to current limitations of our Python/Rust interface,
    # we need to convert the numpy arrays to lists
    q, v = np.array([q]), np.array([v])
    u = np.array(u)

    # apply the ABA algorithm to compute the acceleration
    a = pin.aba(model, data_sim, q, v, u)

    # numpy conversion
    v = np.array([v])
    q = np.array([q])

    # integrate the acceleration to get the new state
    v += a * dt
    q = pin.integrate(model, q, v * dt)

    q, v = list(q)[0], list(v)[0]
    return [q, v]

Q = [[0, 0], [0, 0]] # state cost
Qf = [[1, 0], [0, 1]] # final state cost
R = [[1e-5]] # control cost (minimize the energy)
s = ilqr.ILQRSolver(2, 1, Q, Qf, R)

target = [0, 0] # target state, upright pendulum with no velocity
controls = s.solve([q, v], target, dynamics, time_steps=N)

t = 0.0
for k in range(N):
    tau_control = controls[k]
    q, v = dynamics((q, v), tau_control)
    q, v = q.item(), v.item()

    viz.display(np.array([q]))
    time.sleep(dt)
    t += dt

/var/folders/pr/wxvx7wlx1r5fczqp1p8qmmr80000gp/T/ipykernel_4792/3299812403.py:43: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  controls = s.solve([q, v], target, dynamics, time_steps=N)
